In [ ]:
from fastai.tabular.all import *
from fastai.collab import *
from fastai.data.transforms import *
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
from functools import partial

Developing a recommendation system for a **NO-Q** app, we were focusing on two major scenarios: new users and users who have purchased and rated products in the past.
For users who have purchased and rated products previously, we use their purchase history and ratings to generate personalized recommendations based on collaborative filtering.

**Data**

In user-based filtering, the algorithm identifies users with similar preferences and recommends items that these users have previously enjoyed. To illustrate this approach, we generated a dataset containing 700 users and 441 items rated by users on a scale from 0 to 1 (like/dislike)

The dataset is generated in such a way that some users rated many items while most rated only a few, with some items remaining unrated - a realistic representation of a typical data

In [ ]:
#Load data
df = pd.read_csv('User ratings.csv')
cocktail_strings = df['title'].unique()

In [ ]:
# Create mapping of cocktail strings to integer IDs
cocktail_id_map = {s: i for i, s in enumerate(cocktail_strings)}

In [ ]:
# Replace cocktail strings with their IDs in the dataset
df['cocktail'] = df['title'].map(cocktail_id_map)
df = df[['user','cocktail','rating','title']]
ratings = df

In [ ]:
# Define hyperparameter grid
param_grid = {'lr': [1e-3, 5e-3, 1e-2],
              'wd': [0.01, 0.1, 0.2],
              'n_factors': [20, 50, 100],
              'bs': [32, 64, 128],
              'epochs': [3, 5, 10]}

In [ ]:
# Function to train model with given hyperparameters and return accuracy
def train_model(ratings, param, seed=42):
    torch.manual_seed(seed)
    dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=param['bs'])
    learn = collab_learner(dls, n_factors=param['n_factors'], y_range=(0, 1.0), loss_func=BCEWithLogitsLossFlat())
    learn.fit_one_cycle(param['epochs'], param['lr'], wd=param['wd'])
    
    preds, targets = learn.get_preds()
    accuracy = ((preds>0.5) == targets).float().mean().item()
    return accuracy

In [ ]:
# Grid search
best_accuracy = 0
best_params = None
for param_set in ParameterGrid(param_grid):
    accuracy = train_model(ratings, param_set)
    print(f"Parameters: {param_set}, Accuracy: {accuracy}")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = param_set

print(f"Best Parameters: {best_params}, Best Accuracy: {best_accuracy}")

epoch,train_loss,valid_loss,time
0,0.725321,0.714000,00:01
1,0.728648,0.713720,00:01
2,0.724924,0.713663,00:01


Parameters: {'bs': 32, 'epochs': 3, 'lr': 0.001, 'n_factors': 20, 'wd': 0.01}, Accuracy: 0.5197784900665283


epoch,train_loss,valid_loss,time
0,0.727396,0.714000,00:01
1,0.733498,0.713769,00:01
2,0.722674,0.713698,00:01


Parameters: {'bs': 32, 'epochs': 3, 'lr': 0.001, 'n_factors': 20, 'wd': 0.1}, Accuracy: 0.5197784900665283


epoch,train_loss,valid_loss,time
0,0.729301,0.714054,00:01
1,0.719872,0.713775,00:01
2,0.720957,0.713732,00:01


...

Parameters: {'bs': 128, 'epochs': 10, 'lr': 0.01, 'n_factors': 100, 'wd': 0.01}, Accuracy: 0.5197784900665283


epoch,train_loss,valid_loss,time
0,0.725503,0.713956,00:00
1,0.722932,0.712723,00:00
2,0.715817,0.711562,00:00
3,0.673120,0.711890,00:00
4,0.610959,0.715020,00:00
5,0.566052,0.716816,00:00
6,0.541620,0.717683,00:00
7,0.529163,0.718130,00:00
8,0.522790,0.718328,00:00
9,0.519656,0.718352,00:00


Parameters: {'bs': 128, 'epochs': 10, 'lr': 0.01, 'n_factors': 100, 'wd': 0.1}, Accuracy: 0.5197784900665283


epoch,train_loss,valid_loss,time
0,0.726356,0.713959,00:00
1,0.725331,0.712858,00:00
2,0.716800,0.711153,00:00
3,0.677005,0.711765,00:00
4,0.616462,0.714088,00:00
5,0.572342,0.715509,00:00
6,0.547163,0.716291,00:00
7,0.533456,0.716666,00:00
8,0.526492,0.716828,00:00
9,0.523141,0.716849,00:00


Parameters: {'bs': 128, 'epochs': 10, 'lr': 0.01, 'n_factors': 100, 'wd': 0.2}, Accuracy: 0.5197784900665283
Best Parameters: {'bs': 32, 'epochs': 3, 'lr': 0.001, 'n_factors': 20, 'wd': 0.01}, Best Accuracy: 0.5197784900665283


In [ ]:
# Train the model with the best hyperparameters
best_dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=best_params['bs'])
best_learn = collab_learner(best_dls, n_factors=best_params['n_factors'], y_range=(0, 1.0), loss_func=BCEWithLogitsLossFlat())
best_learn.fit_one_cycle(best_params['epochs'], best_params['lr'], wd=best_params['wd'])


epoch,train_loss,valid_loss,time
0,0.722129,0.729327,00:01
1,0.724421,0.728981,00:01
2,0.716169,0.728909,00:01


In [ ]:

# Menu
items_df = pd.DataFrame({'user': 10,
                      'title': ['Bluebird', 'Boston Sour', 'Irish Spring', 'Zipperhead', 'ACID', 'Spritz', 'Kir Royale', 'Affair', 'Lord and Lady', 'Van Vleet']})  # positions on the menu


In [ ]:

# Learner to get predictions for the items (the probability of liking the item)
preds, _ = best_learn.get_preds(dl=best_dls.test_dl(items_df))

# Get the index of the item with the highest predicted probability
most_likely_item_idx = np.argmax(preds)
most_likely_item_idx = most_likely_item_idx.item()
most_likely_item_title = items_df.loc[most_likely_item_idx, 'title']
print(f"Recommended for you: {most_likely_item_title}")

g = ratings.groupby('title')['rating'].count()
top_cocktails = g.sort_values(ascending=False).index.values[:1000]
top_cocktails[:10]

cocktail_bias = best_learn.model.bias(top_cocktails, is_item=True)
cocktail_bias.shape

mean_ratings = ratings.groupby('title')['rating'].mean()
cocktail_ratings = [(b, i, mean_ratings.loc[i]) for i, b in zip(top_cocktails, cocktail_bias)]

item0 = lambda o: o[0]
items_titles = items_df['title'].unique()

# Filter cocktail_ratings to only include ratings for items on the menu
filtered_cocktail_ratings = [(b, i, mean_ratings.loc[i]) for b, i, r in cocktail_ratings if i in items_titles]
list = sorted(filtered_cocktail_ratings, key=lambda o: o[0], reverse=True)[0:1]
print(f"Top drink on the menu: {list[0][1]}")

Recommended for you: ACID
Top drink on the menu: ACID
